In [1]:
%run bin/pc4nanobio.py
display(gui)

<IPython.core.display.Javascript object>

VBox(children=(Tab(children=(VBox(children=(HBox(children=(Button(button_style='info', description='Use defaults', style=ButtonStyle(), tooltip='Populate the GUI with default parameters'), Button(button_style='info', description='Read config (in ~)', style=ButtonStyle(), tooltip='Populate the GUI with info in your configuration file (in your home directory)'), Text(value='my_nanobio_settings.xml'))), Label(value='Domain ($\\mu M$):'), HBox(children=(FloatText(value=0.0, description='$X_{min}$', layout=Layout(width='180px')), FloatText(value=0.0, description='$X_{max}$', layout=Layout(width='180px')), BoundedFloatText(value=1.0, description='$X_{delta}$', layout=Layout(width='180px'), min=1.0))), HBox(children=(FloatText(value=0.0, description='$Y_{min}$', layout=Layout(width='180px')), FloatText(value=0.0, description='$Y_{max}$', layout=Layout(width='180px')), BoundedFloatText(value=1.0, description='$Y_{delta}$', layout=Layout(width='180px'), min=1.0))), HBox(children=(FloatText(value=0.0, description='$Z_{min}$', layout=Layout(width='180px')), FloatText(value=0.0, description='$Z_{max}$', layout=Layout(width='180px')), BoundedFloatText(value=1.0, description='$Z_{delta}$', layout=Layout(width='180px'), min=1.0))), Label(value=''), BoundedFloatText(value=0.0, description='$Time_{max}$', layout=Layout(width='180px'), max=100000000.0), BoundedIntText(value=1, description='# threads', layout=Layout(width='180px'), min=1), HBox(children=(BoundedFloatText(value=1.0, description='Tumor Radius', layout=Layout(width='180px'), max=99999.0, min=1.0, step=1.0, style=DescriptionStyle(description_width='initial')), HTMLMath(value='$\\mu M$'))), HBox(children=(Checkbox(value=False, description='SVG', layout=Layout(width='180px')), BoundedIntText(value=1, description='interval', layout=Layout(width='180px'), max=99999999, min=1), Checkbox(value=False, description='Full', layout=Layout(width='180px')), BoundedIntText(value=0, description='interval', disabled=True, layout=Layout(width='180px'), max=99999999)))), layout=Layout(height='500px', overflow_y='scroll', width='800px')), VBox(children=(HBox(children=(Text(value='untreated cancer', description='Cell line name', style=DescriptionStyle(description_width='initial')),)), Label(value='Cycle:'), HBox(children=(BoundedFloatText(value=0.0, description='Max birth rate', layout=Layout(width='200px'), style=DescriptionStyle(description_width='initial')), HTMLMath(value='$\\frac{1}{min}$')), layout=Layout(width='300px')), HBox(children=(HBox(children=(BoundedFloatText(value=0.0, description='$O_2$: Prolif sat', layout=Layout(width='180px')), HTMLMath(value='$mmHg$')), layout=Layout(width='230px')), HBox(children=(BoundedFloatText(value=0.0, description='Prolif thresh', layout=Layout(width='180px')), HTMLMath(value='$mmHg$')), layout=Layout(width='230px')), HBox(children=(BoundedFloatText(value=0.0, description='Ref', layout=Layout(width='180px')), HTMLMath(value='$mmHg$')), layout=Layout(width='230px')))), HBox(children=(HBox(children=(BoundedFloatText(value=0.0, description='$Glc$: Prolif sat', layout=Layout(width='180px'), style=DescriptionStyle(description_width='initial')),), layout=Layout(width='230px')), HBox(children=(BoundedFloatText(value=0.0, description='Prolif thresh', layout=Layout(width='180px')),), layout=Layout(width='230px')), HBox(children=(BoundedFloatText(value=0.0, description='Ref', layout=Layout(width='180px')),), layout=Layout(width='230px')))), Label(value='Necrosis:'), HBox(children=(BoundedFloatText(value=0.0, description='Max rate', layout=Layout(width='180px')), HTMLMath(value='$\\frac{1}{min}$')), layout=Layout(width='230px')), HBox(children=(HBox(children=(BoundedFloatText(value=0.0, description='$O_2$: Thresh', layout=Layout(width='180px')), HTMLMath(value='$mmHg$')), layout=Layout(width='230px')), HBox(children=(BoundedFloatText(value=0.0, description='Max', layout=Layout(width='180px')), HTMLMath(value='$mmHg$')), layout=Layout(width='230px')))), Label(value='Apo